In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

from shared_utils import rt_dates
from segment_speed_utils import helpers, wrangle_shapes
from segment_speed_utils.project_vars import SEGMENT_GCS, SHARED_GCS

analysis_date = rt_dates.DATES["feb2024"]

road_id_cols = ["linearid", "mtfcc", "primary_direction"]
segment_identifier_cols = road_id_cols + ["segment_sequence"]
one_trip = "000015f804849785e168963ae2d91460"
two_roads = [
    "1104259334520",
    "110413812414"
]

In [2]:
vp_nn = gpd.read_parquet(
    f"{SEGMENT_GCS}condensed/vp_nearest_neighbor_{analysis_date}.parquet",
    filters = [[("trip_instance_key", "==", one_trip)]]
)

In [3]:
subset_shape = helpers.import_scheduled_trips(
    analysis_date,
    columns = ["trip_instance_key", "shape_array_key"],
    filters = [[("trip_instance_key", "==", one_trip)]],
    get_pandas = True
)

shapes = helpers.import_scheduled_shapes(
    analysis_date,
    columns = ["shape_array_key", "geometry"],
    filters = [[("shape_array_key", "in", subset_shape.shape_array_key)]],
    crs = "EPSG:3310"
).merge(subset_shape)

shapes = shapes.assign(
    geometry = shapes.geometry.buffer(25)
).to_crs("EPSG:4326")

In [10]:
shapes

,shape_array_key,geometry,trip_instance_key
0,60309c7b742a155be0ec60787bb0705d,"POLYGON ((-122.48475 37.72411, -122.48473 37.7...",000015f804849785e168963ae2d91460


In [21]:
road_segments = gpd.read_parquet(
    f"{SHARED_GCS}road_segments/",
    columns = segment_identifier_cols + ["geometry"],
    filters = [[("linearid", "in", two_roads)]]
)

In [22]:
def sjoin_shape_to_road(shapes, roads):
  
    keep_cols = ["shape_array_key", "trip_instance_key", 
                 "linearid", 
       "mtfcc", "segment_sequence"]
    
    roads = roads.to_crs(shapes.crs)
    
    shapes_to_roads = gpd.sjoin(
        shapes,
        roads,
        how = "inner",
        predicate = "intersects"
    )[keep_cols].drop_duplicates()
    
    return shapes_to_roads

road_segments_sjoin = sjoin_shape_to_road(shapes, road_segments)

In [23]:
road_segments_sjoin

,shape_array_key,trip_instance_key,linearid,mtfcc,segment_sequence
0,60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,1104259334520,S1400,0
0,60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,110413812414,S1400,0


In [25]:
road_segments2 = pd.merge(
    road_segments,
    road_segments_sjoin,
    on = ["linearid", "mtfcc", "segment_sequence"], 
    how = "inner"
)

In [27]:
import shapely

road_segments0 = road_segments2.assign(
    geometry = road_segments2.apply(
        lambda x: shapely.Point(x.geometry.coords[0]), 
        axis=1),
).assign(stop_type=0)

road_segments1 = road_segments2.assign(
    geometry = road_segments2.apply(
        lambda x: shapely.Point(x.geometry.coords[-1]), 
        axis=1),
).assign(stop_type=1)

In [28]:
road_segments_long = pd.concat(
    [road_segments0, road_segments1], 
    axis=0
).sort_values(
    ["linearid", "segment_sequence", "stop_type"]
).rename(
    columns = {"primary_direction": "stop_primary_direction"}
).reset_index(drop=True)

In [30]:
from segment_speed_utils import neighbor

gdf = neighbor.merge_stop_vp_for_nearest_neighbor(
    road_segments_long, 
    analysis_date
)

In [31]:
gdf

,linearid,mtfcc,stop_primary_direction,segment_sequence,stop_geometry,shape_array_key,trip_instance_key,stop_type,vp_geometry,vp_idx
0,110413812414,S1400,Southbound,0,POINT (-122.46955 37.70522),60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,0,"LINESTRING (-122.47057 37.70525, -122.47081 37...","[12811292, 12811294, 12811295, 12811297, 12811..."
1,110413812414,S1400,Southbound,0,POINT (-122.46951 37.70194),60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,1,"LINESTRING (-122.47057 37.70525, -122.47081 37...","[12811292, 12811294, 12811295, 12811297, 12811..."
2,1104259334520,S1400,Southbound,0,POINT (-122.47072 37.70511),60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,0,"LINESTRING (-122.47057 37.70525, -122.47081 37...","[12811292, 12811294, 12811295, 12811297, 12811..."
3,1104259334520,S1400,Southbound,0,POINT (-122.47025 37.69617),60309c7b742a155be0ec60787bb0705d,000015f804849785e168963ae2d91460,1,"LINESTRING (-122.47057 37.70525, -122.47081 37...","[12811292, 12811294, 12811295, 12811297, 12811..."
